In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [4]:
df=pd.read_csv('train.csv')

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [7]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### Train and Test split

In [119]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,1:8],df.iloc[:,0],test_size=0.2,random_state=42)

In [120]:
X_train.shape

(712, 7)

In [121]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

## AGE IMPUTATION

In [122]:
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')



In [123]:
trf2=ColumnTransformer([
    ('one_hot_encoding',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [125]:
#Scaling
trf3=ColumnTransformer([
    ('Scaling',MinMaxScaler(),slice(0,10))
    
])

In [126]:
#trf4=SelectKBest(score_func=chi2,k=10)

In [127]:
trf5=DecisionTreeClassifier()

In [128]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf5',trf5)
    ])

In [129]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encoding',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('Scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf5', DecisionTreeClassifier())])

# Exploring the Pipeline

In [130]:
from sklearn import set_config
set_config(display='diagram')

In [131]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('one_hot_encoding',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('Scaling', MinMaxScaler(),
                                  slice(0, 10, None))]),
 'trf5': DecisionTreeClassifier()}

In [132]:
#To get the transformers used in the pipeline
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [133]:
pipe.named_steps['trf1'].transformers_[0]

('impute_age', SimpleImputer(), [2])

In [134]:
pipe.named_steps['trf1'].transformers_[0][1]

SimpleImputer()

In [135]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [136]:
pipe.named_steps['trf2'].transformers_[0][1].categories_

[array(['C', 'Q', 'S'], dtype=object),
 array([0.0, 4.0125, 5.0, 6.2375, 6.4375, 6.45, 6.4958, 6.75, 6.8583, 6.95,
        6.975, 7.05, 7.0542, 7.125, 7.1417, 7.225, 7.2292, 7.25, 7.3125,
        7.4958, 7.5208, 7.55, 7.6292, 7.65, 7.725, 7.7333, 7.7375, 7.7417,
        7.75, 7.775, 7.7958, 7.8, 7.8292, 7.8542, 7.8792, 7.8875, 7.8958,
        7.925, 8.0292, 8.05, 8.1125, 8.1375, 8.3, 8.3625, 8.4042, 8.5167,
        8.6542, 8.6625, 8.6833, 8.85, 9.0, 9.2167, 9.35, 9.475, 9.4833,
        9.5, 9.5875, 9.825, 9.8375, 9.8417, 10.1708, 10.4625, 10.5,
        10.5167, 11.1333, 11.2417, 11.5, 12.0, 12.275, 12.2875, 12.35,
        12.475, 12.525, 12.65, 13.0, 13.4167, 13.5, 13.7917, 13.8583,
        13.8625, 14.0, 14.1083, 14.4, 14.4542, 14.4583, 14.5, 15.0,
        15.0458, 15.05, 15.1, 15.2458, 15.5, 15.7417, 15.75, 15.85, 15.9,
        16.0, 16.1, 16.7, 17.4, 17.8, 18.0, 18.75, 18.7875, 19.2583, 19.5,
        19.9667, 20.2125, 20.25, 20.525, 20.575, 21.0, 21.075, 21.6792,
        22.025, 22.

In [137]:
y_prd=pipe.predict(X_test)
y_prd
y_test.head()

709    1
439    0
840    0
720    1
39     1
Name: Survived, dtype: int64

In [138]:
from sklearn.metrics import accuracy_score

In [139]:
accuracy_score(y_prd,y_test)

0.6256983240223464

In [140]:
import pickle

## Cross Validation using Pipeline

In [141]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

## Gridsearch CV

In [142]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}


In [143]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('one_hot_encoding',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('Scaling',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [144]:
grid.best_score_

0.6391214419383433

In [145]:
grid.best_params_

{'trf5__max_depth': 2}

In [146]:
#pickle
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))